## Clasificación de personas en situación de pobreza

In [ ]:
# !pip install xgboost
# !pip install tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_curve
)

# Clasificadores
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.random import set_seed

import warnings
warnings.simplefilter("ignore")

In [ ]:
dta = pd.read_csv("./data/data_f.csv", index_col=0)
dta.head()

### Split y estandarización
Primero definimos las variables que tendremos como explicativas y al target

In [ ]:
dta = dta.loc[(dta["ingbrut"] > 0)]
dta["lning"] = np.log(dta["ingbrut"])
X_names = ["edad", "hombre", "est_sup", "colegio_priv", "gran_empresa", "segundo_trab", "urbano", "lning"]
X = dta[X_names]
y = dta["no_pobre"]

# Estandarización
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Visualizaciones

In [ ]:
print(f"La distribución de la variable `no pobre` es:\n{y.value_counts()}\n")
print(f"La distribución de la variable `no pobre` de entrenamiento es:\n{y_train.value_counts()}\n")
print(f"La distribución de la variable `no pobre` de testing es:\n{y_test.value_counts()}\n")

In [ ]:
plt.plot()
correl = pd.DataFrame(X, columns=X_names).corr()

sns.heatmap(correl, annot=True, fmt = ".1f", cmap="RdPu")
plt.xticks(rotation=90)

plt.title("Correlación entre explicativas")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)

axes[0].hist(dta["edad"], bins=20, density=True)
axes[1].hist(dta["lning"], bins=50, density=True)

fig.suptitle("Distribuciones de las edades y del logaritmo de los ingresos")

plt.savefig("./figures/distribuciones.pdf")
plt.show()

### Relaciones entre las variables
Para poder encontrar relaciones entre las variables, debido a que los modelos de clasificación no necesariante tienen ese objetivo. se puede estimar un modelo Logit

In [ ]:
model = sm.Logit(y, X)
results = model.fit()

print(results.summary())

In [ ]:
AME = results.get_margeff(at="overall", method="dydx")

print(AME.summary())

Los resultados son interesantes:

* Las personas con mayor edad tienen menos probabilidad de ser probreza
* El ser hombre no reduce el nivel de pobreza, sino lo vuelve mas probable
* Tener un estudio superior (no universitario, universitario, maestria o doctorado) reduce los niveles de pobreza
* Haber estudiado en un colegio privado reduce la probabilidad de ser pobre
* Vivir en una zona urbana tambien reduce la probabilidad de ser pobre
* _El mas evidente_: recibir mayores ingresos tambien reduce la probabilidad de ser pobre

### Modelo 1: Modelo lineal

In [ ]:
ml = LinearRegression().fit(X_train, y_train)

yhat_proba_ml = ml.predict(X_test)
yhat_ml = (yhat_proba_ml >= 0.5).astype(int)

# Estadisticos
score_ml = accuracy_score(y_test, yhat_ml)
ps_ml = precision_score(y_test, yhat_ml)
r_ml = recall_score(y_test, yhat_ml)
f1_ml = f1_score(y_test, yhat_ml)

# Confusion matrix y ROC
conf_ml = confusion_matrix(y_test, yhat_ml, normalize="true")
report_ml = classification_report(y_test, yhat_ml)
fml, tml, thresholds = roc_curve(y_test, yhat_proba_ml)

print(report_ml)

### Modelo 2. Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=19).fit(X_train, y_train)

yhat_proba_lr = lr.predict_proba(X_test)[:,1]
yhat_lr = lr.predict(X_test)

# Estadisticos
score_lr = accuracy_score(y_test, yhat_lr)
ps_lr = precision_score(y_test, yhat_lr)
r_lr = recall_score(y_test, yhat_lr)
f1_lr = f1_score(y_test, yhat_lr)

# Confusion matrix y ROC
conf_lr = confusion_matrix(y_test, yhat_lr, normalize="true")
report_lr = classification_report(y_test, yhat_lr)
flr, tlr, thresholds = roc_curve(y_test, yhat_proba_lr)

print(report_lr)

### Modelo 3. K-Nearest Neighbours

In [ ]:
# Grid Search: Cantidad de Neigbours
knn_grid_search = KNeighborsClassifier()
k_range = list(range(1, 26))
param_grid = {
    'n_neighbors': k_range,
    'weights': ["uniform"]
}

grid = GridSearchCV(
    knn_grid_search, param_grid, cv = 10, scoring = "accuracy",
    return_train_score = False, verbose = 1,  n_jobs = -1 # Paralelización
)

knn = grid.fit(X_train, y_train)

In [ ]:
print(knn.best_params_)
print(knn.best_score_ )

In [ ]:
yhat_proba_knn = knn.predict_proba(X_test)[:,1]
yhat_knn = knn.predict(X_test)

# Estadisticos
score_knn = accuracy_score(y_test, yhat_knn)
ps_knn = precision_score(y_test, yhat_knn)
r_knn = recall_score(y_test, yhat_knn)
f1_knn = f1_score(y_test, yhat_knn)

# Confusion matrix y ROC
conf_knn = confusion_matrix(y_test, yhat_knn, normalize="true")
report_knn = classification_report(y_test, yhat_knn)
fknn, tknn, thresholds = roc_curve(y_test, yhat_proba_knn)

print(report_knn)

### Modelo 4. Decision Tree

In [ ]:
# Grid Search: Cantidad de Ramas
tree_grid_search = DecisionTreeClassifier()
max_depth_range = list(range(1, 11))
param_grid = {
    'criterion' : ["entropy"],
    'max_depth' : max_depth_range,
    'random_state': [19]
}

grid = GridSearchCV(
    tree_grid_search, param_grid, cv = 10, scoring = "accuracy",
    return_train_score = False, verbose = 1, n_jobs = -1
)

tree = grid.fit(X_train, y_train)

In [ ]:
print(tree.best_params_)
print(tree.best_score_ )

In [ ]:
yhat_proba_tree = tree.predict_proba(X_test)[:,1]
yhat_tree = tree.predict(X_test)

# Estadisticos
score_tree = accuracy_score(y_test, yhat_tree)
ps_tree = precision_score(y_test, yhat_tree)
r_tree = recall_score(y_test, yhat_tree)
f1_tree = f1_score(y_test, yhat_tree)

# Confusion matrix y ROC
conf_tree = confusion_matrix(y_test, yhat_tree, normalize="true")
report_tree = classification_report(y_test, yhat_tree)
ftree, ttree, thresholds = roc_curve(y_test, yhat_proba_tree)

print(report_tree)

### Modelo 5. Random Forest

In [ ]:
# Obteniendo los hyperparameters
rf_grid_search = RandomForestClassifier()
rf_grid_search.get_params()

In [ ]:
# Grid Search: Cantidad de Ramas y Árboles
rf_grid_search = RandomForestClassifier()
max_depth_range = list(range(1, 11))
n_estimators_range = [100, 200, 300, 400, 500]

param_grid = {
    'criterion' : ["entropy"],
    'max_depth': max_depth_range,       # Máxima profundidad
    'n_estimators': n_estimators_range, # Número de arboles
    'random_state': [19]
}

grid = GridSearchCV(
    rf_grid_search, param_grid, cv = 10, scoring = "accuracy",
    return_train_score = False, verbose = 1, n_jobs = -1
)

rf = grid.fit(X_train, y_train)

In [ ]:
print(rf.best_params_)
print(rf.best_score_ )

In [ ]:
yhat_proba_rf = rf.predict_proba(X_test)[:,1]
yhat_rf = rf.predict(X_test)

# Estadisticos
score_rf = accuracy_score(y_test, yhat_rf)
ps_rf = precision_score(y_test, yhat_rf)
r_rf = recall_score(y_test, yhat_rf)
f1_rf = f1_score(y_test, yhat_rf)

# Confusion matrix y ROC
conf_rf = confusion_matrix(y_test, yhat_rf, normalize="true")
report_rf = classification_report(y_test, yhat_rf)
frf, trf, thresholds = roc_curve(y_test, yhat_proba_rf)

print(report_rf)

### Modelo 6. Neural Network

In [ ]:
set_seed(19)

nn = Sequential(
    [
        Input(shape = (8,)),
        Dense(units = 32, activation = "sigmoid", name = "Layer1"),
        Dense(units = 16, activation = "sigmoid", name = "Layer2"),
        Dense(units = 8, activation = "sigmoid", name = "Layer3"),
        Dense(units = 1, activation = "sigmoid", name = "Layer4")
    ]
)

nn.compile(
    loss = BinaryCrossentropy(),
    optimizer = Adam(0.001)
)

results = nn.fit(X_train, y_train, epochs = 50, verbose = False)

yhat_proba_nn = nn.predict(X_test)
yhat_nn = (yhat_proba_nn >= .5).astype(int)

# Estadisticos
score_nn = accuracy_score(y_test, yhat_nn)
ps_nn = precision_score(y_test, yhat_nn)
r_nn = recall_score(y_test, yhat_nn)
f1_nn = f1_score(y_test, yhat_nn)

# Confusion matrix y ROC
conf_nn = confusion_matrix(y_test, yhat_nn, normalize="true")
report_nn = classification_report(y_test, yhat_nn)
fnn, tnn, thresholds = roc_curve(y_test, yhat_proba_nn)

print(report_nn)

In [ ]:
plt.figure()

plt.plot(results.history["loss"], color="black")

plt.title("Función de pérdida para cada iteración", fontsize = 10)

plt.savefig("./figures/loss-function.pdf")
plt.show()

### Modelo 7. Support Vector Machine

In [ ]:
# Grid Search:
svm_grid_search = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf'],
    'random_state': [19]
}

grid = GridSearchCV(
    svm_grid_search, param_grid, cv = 10, scoring = "accuracy",
    return_train_score = False, verbose = 1, n_jobs = -1
)

svm = grid.fit(X_train, y_train)

In [ ]:
print(svm.best_params_)
print(svm.best_score_ )

In [ ]:
yhat_proba_svm = svm.predict_proba(X_test)[:,1]
yhat_svm = svm.predict(X_test)

# Estadisticos
score_svm = accuracy_score(y_test, yhat_svm)
ps_svm = precision_score(y_test, yhat_svm)
r_svm = recall_score(y_test, yhat_svm)
f1_svm = f1_score(y_test, yhat_svm)

# Confusion matrix y ROC
conf_svm = confusion_matrix(y_test, yhat_svm, normalize="true")
report_svm = classification_report(y_test, yhat_svm)
fsvm, tsvm, thresholds = roc_curve(y_test, yhat_proba_svm)

print(report_svm)

### Modelo 8. XGBoost

In [ ]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric = "mlogloss", random_state = 19).fit(X_train, y_train)

yhat_proba_xgb = xgb.predict_proba(X_test)[:,1]
yhat_xgb = xgb.predict(X_test)

# Estadisticos
score_xgb = accuracy_score(y_test, yhat_xgb)
ps_xgb = precision_score(y_test, yhat_xgb)
r_xgb = recall_score(y_test, yhat_xgb)
f1_xgb = f1_score(y_test, yhat_xgb)

# Confusion matrix y ROC
conf_xgb = confusion_matrix(y_test, yhat_xgb, normalize="true")
report_xgb = classification_report(y_test, yhat_xgb)
fxgb, txgb, thresholds = roc_curve(y_test, yhat_proba_xgb)

print(report_xgb)

### Modelo 9. Naive Bayes

In [ ]:
nb_grid_search = GaussianNB()

param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100),
    'random_state': [19]
}

grid = GridSearchCV(
    nb_grid_search, param_grid, cv = 10, scoring = "accuracy", 
    neturn_train_score = False, verbose = 1, n_jobs = -1
)

nb = grid.fit(X_train, y_train)

In [ ]:
print(nb.best_params_)
print(nb.best_score_ )

In [ ]:
yhat_proba_nb = nb.predict_proba(X_test)[:,1]
yhat_nb = nb.predict(X_test)

# Estadisticos
score_nb = accuracy_score(y_test, yhat_nb)
ps_nb = precision_score(y_test, yhat_nb)
r_nb = recall_score(y_test, yhat_nb)
f1_nb = f1_score(y_test, yhat_nb)

# Confusion matrix y ROC
conf_nb = confusion_matrix(y_test, yhat_nb, normalize="true")
report_nb = classification_report(y_test, yhat_nb)
fnb, tnb, thresholds = roc_curve(y_test, yhat_proba_nb)

print(report_nb)

### Comparativa entre modelos

In [ ]:
# 1. Indicadores de ajuste en testing
dict_ajust = {
    "Regresión lineal": score_ml,
    "Regresión logistica": score_lr,
    "KNN": score_knn,
    "Decision tree": score_tree,
    "Random forest": score_rf,
    "Neural networks": score_nn,
    "SVM": score_svm,
    "XGBoost": score_xgb,
    "Naive Bayes": score_nb
}

df_score = pd.DataFrame(dict_ajust.items(), columns=["Modelo", "Score"]).set_index("Modelo")
df_score["Score"] = np.round(df_score["Score"], 3)
df_score = df_score.sort_values("Score", ascending=False)

df_score

In [ ]:
fig, ax = utils.render_mpl_table(df_score.reset_index(), header_columns=0, col_width=2.0)
fig.savefig("./figures/score.png")

In [ ]:
# 2. Precision
dict_precision = {
    "Regresión lineal": ps_ml,
    "Regresión logistica": ps_lr,
    "KNN": ps_knn,
    "Decision tree": ps_tree,
    "Random forest": ps_rf,
    "Neural networks": ps_nn,
    "SVM": ps_svm,
    "XGBoost": ps_xgb,
    "Naive Bayes": ps_nb
}

df_precision = pd.DataFrame(dict_precision.items(), columns=["Modelo", "Precision"]).set_index("Modelo")
df_precision["Precision"] = np.round(df_precision["Precision"], 3)
df_precision = df_precision.sort_values("Precision", ascending=False)

df_precision

In [ ]:
# 3. Recall
dict_recall = {
    "Regresión lineal": r_ml,
    "Regresión logistica": r_lr,
    "KNN": r_knn,
    "Decision tree": r_tree,
    "Random forest": r_rf,
    "Neural networks": r_nn,
    "SVM": r_svm,
    "XGBoost": r_xgb,
    "Naive Bayes": r_nb
}

df_recall = pd.DataFrame(dict_recall.items(), columns=["Modelo", "Recall"]).set_index("Modelo")
df_recall["Recall"] = np.round(df_recall["Recall"], 3)
df_recall = df_recall.sort_values("Recall", ascending=False)

df_recall

In [ ]:
# 4. F1-Score
dict_f1 = {
    "Regresión lineal": f1_ml,
    "Regresión logistica": f1_lr,
    "KNN": f1_knn,
    "Decision tree": f1_tree,
    "Random forest": f1_rf,
    "Neural networks": f1_nn,
    "SVM": f1_svm,
    "XGBoost": f1_xgb,
    "Naive Bayes": f1_nb
}

df_f1 = pd.DataFrame(dict_f1.items(), columns=["Modelo", "F1-Score"]).set_index("Modelo")
df_f1["F1-Score"] = np.round(df_f1["F1-Score"], 3)
df_f1 = df_f1.sort_values("F1-Score", ascending=False)

df_f1

In [ ]:
# 5. Matriz de confusion
dict_conf = {
    "Regresión lineal": conf_ml,
    "Regresión logistica": conf_lr,
    "KNN": conf_knn,
    "Decision tree": conf_tree,
    "Random forest": conf_rf,
    "Neural networks": conf_nn,
    "SVM": conf_svm,
    "XGBoost": conf_xgb,
    "Naive Bayes": conf_nb
}

conf_names = list(dict_conf.keys())
conf_matrixes = dict_conf.values()


# Figura
fig, ax = plt.subplots(3, 3, figsize=(12, 12))

r = 0; c = 0; i = 0

for matrix in conf_matrixes:
    fig = ConfusionMatrixDisplay(
        confusion_matrix = matrix,
    ).plot(ax = ax[r, c], colorbar = False, cmap=plt.cm.Blues)
    
    ax[r, c].set_title(conf_names[i])
    ax[r, c].set_xlabel("Predicción")
    ax[r, c].set_ylabel("Real")
    
    i += 1
    r += 1
    if r == 3:
        c += 1
        r = 0
    else:
        pass

plt.suptitle("Matrices de confusión (normalizadas a los valores reales)")
plt.subplots_adjust(top=0.9, bottom=0.4, left=0.2, right=0.75, hspace=0.6, wspace=0.5)

plt.savefig("./figures/matrices_confusion.pdf", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
# 6. ROC
dict_conf = {
    "Regresión lineal": [fml, tml],
    "Regresión logistica": [flr, tlr],
    "KNN": [fknn, tknn],
    "Decision tree": [ftree, ttree],
    "Random forest": [frf, trf],
    "Neural networks": [fnn, tnn],
    "SVM": [fsvm, tsvm],
    "XGBoost": [fxgb, txgb],
    "Naive Bayes": [fnb, tnb]
}
keys = list(dict_conf.keys())
values = list(dict_conf.values())


# Figura
plt.figure(figsize=(6, 5))
plt.plot([0, 1], [0, 1], "k--", label="(ROC = 0.5)")

j = 0
for i in values:
    plt.plot(i[0], i[1], label=keys[j])
    j += 1

plt.xlabel("Ratio falso positivo")
plt.ylabel("Ratio verdadero positivo")
plt.title("Curvas ROC")
plt.legend(fontsize=9)

plt.savefig("./figures/roc.pdf", bbox_inches="tight", transparent=True)
plt.savefig("./figures/roc.png", bbox_inches="tight", transparent=True)
plt.show()